### 가사 크롤링

In [1]:
!pip install beautifulsoup4

#### base file 수정 -> filtered_sim_lyrics_revice.csv 만들기

In [3]:
import csv

base_file = r'../data_v2/filtered_sim_lyrics.csv'

with open(base_file, mode="r", newline="", encoding="utf-8") as basefile:
    base = list(csv.reader(basefile))
    
import pandas as pd

# 첫 번째 행을 컬럼명으로 사용
df = pd.DataFrame(base[1:], columns=base[0])  # 첫 번째 행을 컬럼명으로 지정

# print(df.head())  

output_file = r'../data_v2/filtered_sim_lyrics_revice.csv'

df.to_csv(output_file, index=False, quoting=csv.QUOTE_ALL, encoding="utf-8")

#### base file (filterd_sim_lyrics_revice.csv)파일 읽어오기

In [2]:
import csv

base_file = r'../data_v2/filtered_sim_lyrics_revice.csv'

with open(base_file, mode="r", newline="", encoding="utf-8") as basefile:
    base = list(csv.reader(basefile))

In [3]:
import pandas as pd

# 첫 번째 행을 컬럼명으로 사용
df = pd.DataFrame(base[1:], columns=base[0])  # 첫 번째 행을 컬럼명으로 지정

print(df.head())  

                  genre            track_id       artist_name  \
0  classic pop and rock  TRWUBTF128F4278A66  Blue Oyster Cult   
1  classic pop and rock  TRMJRWO128F4267A78  Blue Oyster Cult   
2  classic pop and rock  TRMMSXG128F4260B30  Blue Oyster Cult   
3  classic pop and rock  TRWTLZC128F4260B56  Blue Oyster Cult   
4  classic pop and rock  TRWWXBZ128F92C2C0D  Blue Oyster Cult   

                                  title  
0                   The Red & The Black  
1                             Vengeance  
2  I'm On The Lamb But I Ain't No Sheep  
3                         Buck's Boogie  
4                     Kick Out The Jams  


In [4]:
title = df['title']
artist = df['artist_name']

#### genius.com 크롤링

In [5]:
import re

def remove_special_characters_title(title):
    text = title.lower()
    text = text.replace('/', '&')   # / → &
    text = text.replace('&', 'and')  # & → and
    text = text.replace("'", '')  # 작은따옴표 제거
    text = text.replace('=', '-')  # = → -
    text = re.sub(r"\(.*?\)", "", text)  # 괄호 안의 내용 포함 삭제
    text = re.sub(r"[?!.,/@#]", "", text)  # 불필요한 특수문자 제거
    text = re.sub(r"\s+", "-", text)  # 공백 → 대시 (-) 변환
    return text.strip("-")  # 앞뒤 불필요한 '-' 제거

def remove_special_characters_artist(artist):
    text = artist.lower()
    text = text.replace('/', '&')   # / → &
    text = text.replace('&', 'and')  # & → and
    text = text.replace("'", '')  # 작은따옴표 제거
    text = text.replace('=', '-')  # = → -
    text = re.sub(r"\(.*?\)", "", text)  # 괄호 안의 내용 포함 삭제
    text = re.sub(r"[?!.,/@#]", "", text)  # 불필요한 특수문자 제거
    text = re.sub(r"\s+", "-", text)  # 공백 → 대시 (-) 변환
    return text.strip("-")  # 앞뒤 불필요한 '-' 제거

def format_artist_title(base_path, artist, title):
    formatted_artist = artist.capitalize()
    formatted_artist = remove_special_characters_artist(formatted_artist)
    
    formatted_title = title.lower()
    formatted_title = remove_special_characters_title(formatted_title)
    
    url = base_path + formatted_artist + '-' + formatted_title + '-lyrics'
    return url

In [6]:
# 체크포인트 파일이 존재하면 읽고, 존재하지 않으면 초기화
import os
import json

checkpoint_file = 'lyrics_genius_checkpoint.json'

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        try:
            checkpoint = json.load(f)
        except json.JSONDecodeError:
            # JSONDecodeError 발생 시 기본값으로 초기화
            checkpoint = {'last_processed_index': 0, 'error_num': 0}
            lyrics_all = []
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f)
    start_index = checkpoint.get('last_processed_index', 0)
    error_num = checkpoint.get('error_num', 0)
    lyrics_all = checkpoint.get('lyrics_all', []) 
else:
    start_index = 0
    error_num = 0
    lyrics_all = []

In [7]:
# checkpoint 확인용
print(start_index, error_num, len(lyrics_all))
lyrics_none_index = [n for n,lyric in enumerate(lyrics_all) if lyric == ""]
print(lyrics_none_index)

4317 1568 4317
[1, 3, 9, 10, 16, 21, 23, 24, 32, 33, 34, 35, 36, 37, 43, 57, 66, 68, 70, 76, 77, 85, 90, 99, 102, 103, 104, 105, 107, 109, 111, 121, 131, 136, 139, 141, 142, 145, 148, 153, 154, 155, 157, 159, 160, 176, 177, 181, 185, 189, 195, 197, 199, 203, 204, 206, 207, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 237, 238, 247, 248, 249, 272, 275, 276, 286, 290, 293, 296, 302, 313, 322, 323, 324, 339, 341, 352, 365, 367, 371, 373, 375, 378, 379, 380, 381, 384, 385, 386, 387, 390, 391, 392, 393, 394, 400, 401, 408, 409, 410, 411, 413, 414, 415, 416, 439, 441, 442, 443, 444, 445, 446, 449, 450, 451, 455, 458, 459, 462, 470, 471, 472, 474, 475, 477, 478, 479, 486, 487, 488, 492, 493, 494, 495, 496, 498, 499, 507, 514, 515, 516, 517, 519, 531, 532, 533, 534, 535, 537, 538, 539, 540, 541, 542, 543, 545, 555, 559, 560, 565, 574, 577, 585, 587, 590, 592, 595, 602, 603, 619, 624, 625, 633, 648, 650, 651, 662, 663, 664, 665, 666, 668, 672, 673, 675, 676, 681, 682, 6

In [20]:
import requests
from bs4 import BeautifulSoup
url = 'https://genius.com/Andrew-gold-go-back-home-again-lyrics'
response = requests.get(url)

html = response.text
soup = BeautifulSoup(html, 'html.parser')

# 가사 부분 찾기 (data-lyrics-container="true" 속성 사용)
lyrics_div = soup.find("div", {"data-lyrics-container": "true"})

if lyrics_div:
    lyrics = lyrics_div.get_text(separator=" ", strip=True)  # 띄어쓰기기로 구분하여 텍스트 추출
    # print(lyrics)
    
    #[] 내부의 내용 제거 : chorus, verse, bridge 등의 정보 삭제
    lyrics = re.sub(r"\[.*?\]", "", lyrics).strip()

print(lyrics)
    

Well, there ain't no use to tell you how I'm doing 'Cause you don't really care at all You're just feelin' guilty It's not my job to repair So just go back home And cry the end It's the same old thing That I did my friend There's just no good reason For a hasty mend So just go back home again Well, the thing fell through so many times before That I felt like a fool And you make no effort to retrieve all the pieces And you broke the golden rule So just go back home And cry the end It's the same old thing That I did my friend There's just no good reason For a hasty mend So just go back home again


In [8]:
import requests
from bs4 import BeautifulSoup

base_path = 'https://genius.com/'

for index in range(start_index,len(title)): 
    url = format_artist_title(base_path, artist[index], title[index])
    # print(url)
    
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 가사 부분 찾기 (data-lyrics-container="true" 속성 사용)
        lyrics_div = soup.find("div", {"data-lyrics-container": "true"})
        
        if lyrics_div:
            lyrics = lyrics_div.get_text(separator=" ", strip=True)  # 띄어쓰기기로 구분하여 텍스트 추출
            # print(lyrics)
            
            #[] 내부의 내용 제거 : chorus, verse, bridge 등의 정보 삭제
            lyrics = re.sub(r"\[.*?\]", "", lyrics).strip()
            
        else: # genius.com에서 노래는 있지만, 가사는 찾을 수 없는 경우
            lyrics = None
            print(url)
            print("가사를 찾을 수 없습니다.")
            error_num += 1

    else: # url 자체가 존재하지 않는 경우
        lyrics = None
        print(url)
        print(f"에러 발생: {response.status_code}")
        error_num += 1
        
    # 다운로드가 성공적으로 완료되면 체크포인트 업데이트
    # df.loc[index, 'lyrics'] = lyrics if lyrics is not None else ""
    lyrics_all.append(lyrics if lyrics is not None else "")
    
    checkpoint = {'last_processed_index': index + 1, 
                  'error_num' : error_num, 
                  'lyrics_all' : lyrics_all}
    with open(checkpoint_file, 'w') as f:
        json.dump(checkpoint, f)

https://genius.com/john-stevens-here_-there-and-everywhere-lyrics
에러 발생: 404
https://genius.com/azymuth-voo-sobre-o-horizonte-lyrics
가사를 찾을 수 없습니다.
https://genius.com/john-abercrombie_-dave-holland_-jack-dejohnette-back-woods-song-lyrics
에러 발생: 404
https://genius.com/paddy-milner-unsquare-dance-lyrics
에러 발생: 404
https://genius.com/paddy-milner-run-for-cover-lyrics
에러 발생: 404
https://genius.com/hank-jones-round-midnight-lyrics
에러 발생: 404
https://genius.com/hank-jones-blue-monk-lyrics
에러 발생: 404
https://genius.com/hank-jones-a-child-is-born-lyrics
가사를 찾을 수 없습니다.
https://genius.com/hank-jones-ruby_-my-dear-lyrics
에러 발생: 404
https://genius.com/hank-jones-monks-mood-lyrics
에러 발생: 404
https://genius.com/hank-jones-walidi-ya-lyrics
에러 발생: 404
https://genius.com/hank-jones-moriba-ka-foly-lyrics
에러 발생: 404
https://genius.com/hank-jones-komidiara-lyrics
에러 발생: 404
https://genius.com/charlie-haden-la-passionaria-lyrics
에러 발생: 404
https://genius.com/charlie-haden-my-foolish-heart-lyrics
에러 발생: 404

In [12]:
# checkpoint 확인용
print(checkpoint['last_processed_index'], checkpoint['error_num'], len(checkpoint['lyrics_all']))

# lyrics_all에 가사가 없는 경우의 인덱스를 찾아서 none_index에 저장
lyrics_none_index = [n for n,lyric in enumerate(lyrics_all) if lyric == ""]
lyrics_filled_index = [n for n,lyric in enumerate(lyrics_all) if lyric != ""]
print(lyrics_none_index)
print(lyrics_filled_index)

print(len(lyrics_none_index))
print(len(lyrics_filled_index))
print(len(lyrics_none_index) + len(lyrics_filled_index))

5045 1972 5045
[1, 3, 9, 10, 16, 21, 23, 24, 32, 33, 34, 35, 36, 37, 43, 57, 66, 68, 70, 76, 77, 85, 90, 99, 102, 103, 104, 105, 107, 109, 111, 121, 131, 136, 139, 141, 142, 145, 148, 153, 154, 155, 157, 159, 160, 176, 177, 181, 185, 189, 195, 197, 199, 203, 204, 206, 207, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 237, 238, 247, 248, 249, 272, 275, 276, 286, 290, 293, 296, 302, 313, 322, 323, 324, 339, 341, 352, 365, 367, 371, 373, 375, 378, 379, 380, 381, 384, 385, 386, 387, 390, 391, 392, 393, 394, 400, 401, 408, 409, 410, 411, 413, 414, 415, 416, 439, 441, 442, 443, 444, 445, 446, 449, 450, 451, 455, 458, 459, 462, 470, 471, 472, 474, 475, 477, 478, 479, 486, 487, 488, 492, 493, 494, 495, 496, 498, 499, 507, 514, 515, 516, 517, 519, 531, 532, 533, 534, 535, 537, 538, 539, 540, 541, 542, 543, 545, 555, 559, 560, 565, 574, 577, 585, 587, 590, 592, 595, 602, 603, 619, 624, 625, 633, 648, 650, 651, 662, 663, 664, 665, 666, 668, 672, 673, 675, 676, 681, 682, 6

In [13]:
df['lyrics'] = lyrics_all
print(df.head())

                  genre            track_id       artist_name  \
0  classic pop and rock  TRWUBTF128F4278A66  Blue Oyster Cult   
1  classic pop and rock  TRMJRWO128F4267A78  Blue Oyster Cult   
2  classic pop and rock  TRMMSXG128F4260B30  Blue Oyster Cult   
3  classic pop and rock  TRWTLZC128F4260B56  Blue Oyster Cult   
4  classic pop and rock  TRWWXBZ128F92C2C0D  Blue Oyster Cult   

                                  title  \
0                   The Red & The Black   
1                             Vengeance   
2  I'm On The Lamb But I Ain't No Sheep   
3                         Buck's Boogie   
4                     Kick Out The Jams   

                                              lyrics  
0  Canadian Mounted , baby Police force at work R...  
1                                                     
2  Canadian Mounted, baby Police force at work Re...  
3                                                     
4  Well, I feel pretty good And I guess that I co...  


In [23]:
df.shape

(5045, 5)

In [ ]:
# checkpoint json -> cvs로 변환
lyrics_file = r'../data_v2/adding_lyrics.csv'
df.to_csv(lyrics_file, index=False, quoting=csv.QUOTE_ALL, encoding="utf-8-sig")

#### musixmatch 크롤링
- 검색창에 제목+아티스트 검색해서 best result의 노래 가사 가져오는 방식 (동적 크롤링)

In [5]:
!pip install selenium webdriver-manager

In [6]:
#필요 라이브러리 설치
#!pip install selenium 

#selenium webdriver 관련 모듈
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup #크롤링한 정보를 파싱하고 정리하기 위한 모듈.
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm #시간별 진행상황을 보기 위한 모듈
import re #정규식
import os
from urllib import request
from urllib.error import HTTPError
import time 
from time import sleep
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
warnings.filterwarnings("ignore")
import random

In [13]:
# 체크포인트 파일이 존재하면 읽고, 존재하지 않으면 초기화
import os
import json

checkpoint_file = 'lyrics_crawling_checkpoint.json'

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        try:
            checkpoint = json.load(f)
        except json.JSONDecodeError:
            # JSONDecodeError 발생 시 기본값으로 초기화
            checkpoint = {'last_processed_index': 0, 'error_num': 0}
            lyrics_all = []
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f)
    start_index = checkpoint.get('last_processed_index', 0)
    error_num = checkpoint.get('error_num', 0)
    lyrics_all = checkpoint.get('lyrics_all', []) 
else:
    start_index = 0
    error_num = 0
    lyrics_all = []

In [19]:
# checkpoint 확인용
print(start_index, error_num, len(lyrics_all))

# lyrics_all에 가사가 없는 경우의 인덱스를 찾아서 none_index에 저장
lyrics_none_index = [n for n,lyric in enumerate(lyrics_all) if lyric == ""]
lyrics_filled_index = [n for n,lyric in enumerate(lyrics_all) if lyric != ""]
print(lyrics_none_index)
print(lyrics_filled_index)

2780 2127 2780
[250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446,

In [20]:
lyrics_all[1119]

'intro\nPsst, I see dead people\nMustard on the beat, ho\nverse\nAy, Mustard on the beat, ho\nDeebo, any rap nigga, he a free-throw\nMan down, call an amberlamps, tell him, "Breathe, bro"\nNail a nigga to the cross, he walk around like Teezo\nWhat′s up with these jabroni-ass niggas tryna see Compton?\nThe industry can hate me, fuck \'em all and they mama\nHow many opps you really got? I mean, it′s too many options\nI\'m finna pass on this body, I\'m John Stockton\nBeat your ass and hide the Bible if God watchin′\nSometimes you gotta pop out and show niggas\nCertified boogeyman, I′m the one that upped the score with \'em\nWalk him down, whole time I know he got some ho in him\nPole on him, extort shit, bully the flow on him\nSay, Drake, I hear you like ′em young\nYou better not ever go to cell block one\nTo any bitch that talk to him and they in love\nJust make sure you hide your lil\' sister from him\nThey tell me Chubbs the only one that get your hand-me-downs\nAnd PARTY at the party,

In [9]:
options=Options()
driver =webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [10]:
import os
import re
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException

driver.implicitly_wait(5)
url = "https://www.musixmatch.com/search"
driver.get(url)

for idx in range(start_index, len(title)):
    full_lyrics = None # 가사가 없는 경우 None으로 초기화
    
    try:
        driver.implicitly_wait(5)
        url = "https://www.musixmatch.com/search"
        driver.get(url)

        search_box = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div/div[1]/div/div/div/div[2]/div/div[1]/div/div/input')
        search_box.clear()
        search_box.send_keys(title[idx] + " " + artist[idx])  # Input the search
        search_box.send_keys(Keys.ENTER)  # Press the search button
        time.sleep(2)
        
        song = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[1]/div/div/div/div[2]/div/div[2]/div[2]/a')
        
        if song:
            href = song[0].get_attribute('href')
            driver.get(href)
            time.sleep(2)
            
            # (1) 가사 전체를 포함하는 div 태그 찾기 (두번째 ~ 마지막에서 두번째 div 태그)
            lyrics_divs = driver.find_elements(By.XPATH, 
                                              '//*[@id="__next"]/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div/div/div[2]/div/*[position() > 1 and position() < last() and not(contains(text(), "Writer(s):"))]')

            # lyrics_divs = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div/div/div[2]/div/*') #모든 태그그

            # (2) 모든 div 태그에서 텍스트 추출
            lyrics = [div.text for div in lyrics_divs if div.text.strip()]
            print('# of lyrics parts:', len(lyrics))
            
            # (3) 리스트를 하나의 문자열로 합치기 (줄바꿈 추가)
            full_lyrics = "\n".join(lyrics)
            print('# of lyrics:', len(full_lyrics))
            
        else: # musixmatch.com에서 노래를 찾을 수 없는 경우
            print(f"{title[idx]} - {artist[idx]}: 노래를 찾을 수 없습니다.")
            error_num += 1
            
    except Exception as e:
        print(f"{title[idx]} - {artist[idx]}: 에러 발생 - {e}")
        error_num += 1
        
    # 다운로드가 성공적으로 완료되면 체크포인트 업데이트
    lyrics_all.append(full_lyrics if full_lyrics is not None else "")
    
    checkpoint = {'last_processed_index': idx + 1, 
                  'error_num' : error_num, 
                  'lyrics_all' : lyrics_all}
    
    with open(checkpoint_file, 'w') as f:
        json.dump(checkpoint, f)

# of lyrics parts: 1
# of lyrics: 624
# of lyrics parts: 7
# of lyrics: 875
# of lyrics parts: 8
# of lyrics: 1935
# of lyrics parts: 10
# of lyrics: 1079
# of lyrics parts: 8
# of lyrics: 1935
# of lyrics parts: 11
# of lyrics: 2070
# of lyrics parts: 9
# of lyrics: 1261
# of lyrics parts: 5
# of lyrics: 950
# of lyrics parts: 7
# of lyrics: 875
# of lyrics parts: 5
# of lyrics: 881
# of lyrics parts: 1
# of lyrics: 1242
# of lyrics parts: 15
# of lyrics: 1525
# of lyrics parts: 2
# of lyrics: 1079
# of lyrics parts: 6
# of lyrics: 686
# of lyrics parts: 7
# of lyrics: 892
# of lyrics parts: 1
# of lyrics: 679
# of lyrics parts: 6
# of lyrics: 1306
# of lyrics parts: 4
# of lyrics: 912
# of lyrics parts: 8
# of lyrics: 860
# of lyrics parts: 6
# of lyrics: 930
# of lyrics parts: 1
# of lyrics: 521
# of lyrics parts: 1
# of lyrics: 409
# of lyrics parts: 1
# of lyrics: 561
# of lyrics parts: 1
# of lyrics: 399
# of lyrics parts: 6
# of lyrics: 930
# of lyrics parts: 1
# of lyrics: 1242

KeyboardInterrupt: 

In [11]:
print(checkpoint['last_processed_index'], checkpoint['error_num'], len(checkpoint['lyrics_all']))

2780 2127 2780
